# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 26 2022 8:30AM,249387,10,MTR-22-0265,"Senseonics, Incorporated",Released
1,Oct 25 2022 4:12PM,249390,21,412958647,"NBTY Global, Inc.",Released
2,Oct 25 2022 4:08PM,249389,21,412958207,"NBTY Global, Inc.",Released
3,Oct 25 2022 4:02PM,249388,21,412957861,"NBTY Global, Inc.",Released
4,Oct 25 2022 3:48PM,249386,21,631420,"NBTY Global, Inc.",Released
5,Oct 25 2022 3:47PM,249385,21,631419,"NBTY Global, Inc.",Released
6,Oct 25 2022 3:37PM,249382,19,TR-WIP-10282022TRK2,"GCH Granules USA, Inc.",Released
7,Oct 25 2022 3:26PM,249379,10,MSP215206,Methapharm-G,Released
8,Oct 25 2022 3:26PM,249379,10,MSP215217,"Methapharm, Inc.",Executing
9,Oct 25 2022 3:26PM,249379,10,MSP215226,"Methapharm, Inc.",Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,249386,Released,1
20,249387,Released,1
21,249388,Released,1
22,249389,Released,1
23,249390,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249386,NaN,NaN,1.0
249387,NaN,NaN,1.0
249388,NaN,NaN,1.0
249389,NaN,NaN,1.0
249390,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249299,14.0,20.0,9.0
249301,0.0,0.0,1.0
249302,0.0,0.0,7.0
249315,0.0,1.0,0.0
249344,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249299,14,20,9
249301,0,0,1
249302,0,0,7
249315,0,1,0
249344,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249299,14,20,9
1,249301,0,0,1
2,249302,0,0,7
3,249315,0,1,0
4,249344,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249299,14,20,9
1,249301,,,1
2,249302,,,7
3,249315,,1,
4,249344,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 26 2022 8:30AM,249387,10,"Senseonics, Incorporated"
1,Oct 25 2022 4:12PM,249390,21,"NBTY Global, Inc."
2,Oct 25 2022 4:08PM,249389,21,"NBTY Global, Inc."
3,Oct 25 2022 4:02PM,249388,21,"NBTY Global, Inc."
4,Oct 25 2022 3:48PM,249386,21,"NBTY Global, Inc."
5,Oct 25 2022 3:47PM,249385,21,"NBTY Global, Inc."
6,Oct 25 2022 3:37PM,249382,19,"GCH Granules USA, Inc."
7,Oct 25 2022 3:26PM,249379,10,Methapharm-G
8,Oct 25 2022 3:26PM,249379,10,"Methapharm, Inc."
11,Oct 25 2022 3:20PM,249376,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 26 2022 8:30AM,249387,10,"Senseonics, Incorporated",,,1
1,Oct 25 2022 4:12PM,249390,21,"NBTY Global, Inc.",,,1
2,Oct 25 2022 4:08PM,249389,21,"NBTY Global, Inc.",,,1
3,Oct 25 2022 4:02PM,249388,21,"NBTY Global, Inc.",,,1
4,Oct 25 2022 3:48PM,249386,21,"NBTY Global, Inc.",,,1
5,Oct 25 2022 3:47PM,249385,21,"NBTY Global, Inc.",,,1
6,Oct 25 2022 3:37PM,249382,19,"GCH Granules USA, Inc.",,,1
7,Oct 25 2022 3:26PM,249379,10,Methapharm-G,,3,1
8,Oct 25 2022 3:26PM,249379,10,"Methapharm, Inc.",,3,1
9,Oct 25 2022 3:20PM,249376,10,ISDIN Corporation,,6,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 26 2022 8:30AM,249387,10,"Senseonics, Incorporated",1,,
1,Oct 25 2022 4:12PM,249390,21,"NBTY Global, Inc.",1,,
2,Oct 25 2022 4:08PM,249389,21,"NBTY Global, Inc.",1,,
3,Oct 25 2022 4:02PM,249388,21,"NBTY Global, Inc.",1,,
4,Oct 25 2022 3:48PM,249386,21,"NBTY Global, Inc.",1,,
5,Oct 25 2022 3:47PM,249385,21,"NBTY Global, Inc.",1,,
6,Oct 25 2022 3:37PM,249382,19,"GCH Granules USA, Inc.",1,,
7,Oct 25 2022 3:26PM,249379,10,Methapharm-G,1,3,
8,Oct 25 2022 3:26PM,249379,10,"Methapharm, Inc.",1,3,
9,Oct 25 2022 3:20PM,249376,10,ISDIN Corporation,,6,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 26 2022 8:30AM,249387,10,"Senseonics, Incorporated",1,,
1,Oct 25 2022 4:12PM,249390,21,"NBTY Global, Inc.",1,,
2,Oct 25 2022 4:08PM,249389,21,"NBTY Global, Inc.",1,,
3,Oct 25 2022 4:02PM,249388,21,"NBTY Global, Inc.",1,,
4,Oct 25 2022 3:48PM,249386,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 26 2022 8:30AM,249387,10,"Senseonics, Incorporated",1.0,NaN,NaN
1,Oct 25 2022 4:12PM,249390,21,"NBTY Global, Inc.",1.0,NaN,NaN
2,Oct 25 2022 4:08PM,249389,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,Oct 25 2022 4:02PM,249388,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Oct 25 2022 3:48PM,249386,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 26 2022 8:30AM,249387,10,"Senseonics, Incorporated",1.0,0.0,0.0
1,Oct 25 2022 4:12PM,249390,21,"NBTY Global, Inc.",1.0,0.0,0.0
2,Oct 25 2022 4:08PM,249389,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,Oct 25 2022 4:02PM,249388,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Oct 25 2022 3:48PM,249386,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1994927,7.0,12.0,0.0
12,249362,0.0,1.0,0.0
15,498601,16.0,20.0,14.0
19,748102,3.0,0.0,0.0
20,498686,1.0,1.0,0.0
21,1496239,6.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1994927,7.0,12.0,0.0
1,12,249362,0.0,1.0,0.0
2,15,498601,16.0,20.0,14.0
3,19,748102,3.0,0.0,0.0
4,20,498686,1.0,1.0,0.0
5,21,1496239,6.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,7.0,12.0,0.0
1,12,0.0,1.0,0.0
2,15,16.0,20.0,14.0
3,19,3.0,0.0,0.0
4,20,1.0,1.0,0.0
5,21,6.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,7.0
1,12,Released,0.0
2,15,Released,16.0
3,19,Released,3.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21
Status,,,,,,
Completed,0.0,0.0,14.0,0.0,0.0,0.0
Executing,12.0,1.0,20.0,0.0,1.0,0.0
Released,7.0,0.0,16.0,3.0,1.0,6.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21
0,Completed,0.0,0.0,14.0,0.0,0.0,0.0
1,Executing,12.0,1.0,20.0,0.0,1.0,0.0
2,Released,7.0,0.0,16.0,3.0,1.0,6.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21
0,Completed,0.0,0.0,14.0,0.0,0.0,0.0
1,Executing,12.0,1.0,20.0,0.0,1.0,0.0
2,Released,7.0,0.0,16.0,3.0,1.0,6.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()